# CSYE 7105 - High Perfoamnce Machine Learning & AI - Project - Team 4 

## Analysis of Efficient Parallel Computing for Deep Learning-Based Glaucoma Detection

### Serial Execution using GPU

#### Importing necessary libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision import transforms
import os
import socket
import time
import json
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from datetime import datetime
import logging
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_recall_fscore_support
os.environ["OMP_NUM_THREADS"] = "1"
torch.set_num_threads(1)

#### Defining the Glaucoma - Medical CNN model from scratch

This MedicalCNN model, with 129 layers, uses a combination of convolutional, residual, multi-dilated, and fully connected blocks, tailored for glaucoma classification. I’ve included Residual Blocks to help with vanishing gradients, MultiDilated Blocks for enhanced feature extraction, and Squeeze-and-Excitation (SE) Blocks to fine-tune feature importance across channels.

In [2]:
class GlaucomaDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        image = self.X[idx]
        label = self.y[idx]
        image = torch.from_numpy(image).float()
        if len(image.shape) == 3:
            image = image.permute(2, 0, 1)  # (H,W,C) -> (C,H,W)
        if self.transform:
            image = self.transform(image)
        return image, label

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, dilation=1):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, 
                              stride=stride, padding=padding, bias=False, dilation=dilation)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=False)
    
    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

class SEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=False),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=False)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.se = SEBlock(out_channels)
        self.downsample = downsample
    
    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.se(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out = out + identity
        out = self.relu(out)
        return out

class MultiDilatedBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(MultiDilatedBlock, self).__init__()
        self.conv1 = ConvBlock(in_channels, out_channels//4, kernel_size=3, padding=1, dilation=1)
        self.conv2 = ConvBlock(in_channels, out_channels//4, kernel_size=3, padding=2, dilation=2)
        self.conv3 = ConvBlock(in_channels, out_channels//4, kernel_size=3, padding=3, dilation=3)
        self.conv4 = ConvBlock(in_channels, out_channels//4, kernel_size=3, padding=4, dilation=4)
        self.conv_fusion = ConvBlock(out_channels, out_channels, kernel_size=1, padding=0)
    
    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        x3 = self.conv3(x)
        x4 = self.conv4(x)
        x = torch.cat([x1, x2, x3, x4], dim=1)
        return self.conv_fusion(x)

class MedicalCNN(nn.Module):
    def __init__(self, num_classes=2, base_filters=64):
        super(MedicalCNN, self).__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, base_filters, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(base_filters),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.stage1 = self._make_stage(base_filters, base_filters, blocks=3)
        self.stage2 = self._make_stage(base_filters, base_filters*2, blocks=4, stride=2)
        self.stage3_res = self._make_stage(base_filters*2, base_filters*4, blocks=6, stride=2)
        self.stage3_md = MultiDilatedBlock(base_filters*4, base_filters*4)
        self.stage4_res = self._make_stage(base_filters*4, base_filters*8, blocks=3, stride=2)
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential(
            nn.Linear(base_filters*8, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=False),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=False),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
        self._initialize_weights()
    
    def _make_stage(self, in_channels, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride, downsample))
        for _ in range(1, blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                if m.weight is not None:
                    nn.init.constant_(m.weight, 1)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3_res(x)
        x = self.stage3_md(x)
        x = self.stage4_res(x)
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    
    def count_layers(self):
        stem_layers = 4
        stage1_layers = 3 * 6
        stage2_layers = 4 * 6
        stage3_layers = 6 * 6 + 10
        stage4_layers = 3 * 6 + 10
        fc_layers = 9
        total_layers = stem_layers + stage1_layers + stage2_layers + stage3_layers + stage4_layers + fc_layers
        return total_layers

def get_medical_transforms():
    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.05, hue=0.05),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    val_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    return train_transform, val_transform


#### Defining the functions to train the model in 1GPU without DDP or other parallelization techniques

In [3]:
# Setting up logging
logging.basicConfig(
    level=logging.INFO,  # Set the logging level to INFO
    format='%(asctime)s - %(levelname)s - %(message)s', 
    handlers=[
        logging.FileHandler("logs/training_using_gpus_1_logs.txt"),
        logging.StreamHandler()  # Also log to console
    ]
)

def train_and_evaluate(data_dir="../../preprocessed_glaucoma_data"):
    # Set random seeds for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    
    logging.info("Loading data...")
    X_train = np.load(os.path.join(data_dir, 'X_train.npy'))
    y_train = np.load(os.path.join(data_dir, 'y_train.npy'))
    X_val = np.load(os.path.join(data_dir, 'X_val.npy'))
    y_val = np.load(os.path.join(data_dir, 'y_val.npy'))
    X_test = np.load(os.path.join(data_dir, 'X_test.npy'))
    y_test = np.load(os.path.join(data_dir, 'y_test.npy'))
    
    logging.info(f"Data shapes: Train {X_train.shape}, Val {X_val.shape}, Test {X_test.shape}")
    
    train_transform, val_transform = get_medical_transforms()
    
    train_dataset = GlaucomaDataset(X_train, y_train, transform=train_transform)
    val_dataset = GlaucomaDataset(X_val, y_val, transform=val_transform)
    test_dataset = GlaucomaDataset(X_test, y_test, transform=val_transform)
    
    # Create data loaders
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    
    unique_classes, class_counts = np.unique(y_train, return_counts=True)
    class_weights = torch.FloatTensor([1.0 / count for count in class_counts])
    class_weights = class_weights / class_weights.sum()
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logging.info(f"Using device: {device}")
    
    num_classes = 2
    model = MedicalCNN(num_classes=num_classes)
    model = model.to(device)
    
    total_params = sum(p.numel() for p in model.parameters())
    total_layers = model.count_layers()
    logging.info(f"Model architecture: MedicalCNN")
    logging.info(f"Total layers: {total_layers}")
    logging.info(f"Total parameters: {total_params:,}")
    
    class_weights = class_weights.to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    initial_lr = 0.001
    weight_decay = 1e-4
    optimizer = optim.AdamW(model.parameters(), lr=initial_lr, weight_decay=weight_decay)
    num_epochs = 10
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=1e-6)
    
    start_time = time.time()
    
    # Lists to record metrics for plotting
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
    
    best_val_acc = 0.0
    best_model_state = None
    
    for epoch in range(num_epochs):
        model.train()
        epoch_train_loss = 0.0
        correct_train = 0
        total_train_samples = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            epoch_train_loss += loss.item() * data.size(0)
            total_train_samples += data.size(0)
            _, predicted = torch.max(output, 1)
            correct_train += (predicted == target).sum().item()
            
            if batch_idx % 10 == 0:
                logging.info(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}')
        
        scheduler.step()
        train_loss = epoch_train_loss / total_train_samples
        train_accuracy = 100.0 * correct_train / total_train_samples
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        logging.info(f'Epoch {epoch+1}/{num_epochs} Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
        
        model.eval()
        epoch_val_loss = 0.0
        correct_val = 0
        total_val_samples = 0
        all_targets = []
        all_predictions = []
        all_probabilities = []
        
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target)
                epoch_val_loss += loss.item() * data.size(0)
                total_val_samples += data.size(0)
                _, predicted = torch.max(output, 1)
                correct_val += (predicted == target).sum().item()
                all_targets.extend(target.cpu().numpy())
                all_predictions.extend(predicted.cpu().numpy())
                all_probabilities.extend(F.softmax(output, dim=1).cpu().numpy())
        
        val_loss = epoch_val_loss / total_val_samples
        val_accuracy = 100.0 * correct_val / total_val_samples
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        
        if num_classes == 2:
            all_targets = np.array(all_targets)
            all_probabilities = np.array(all_probabilities)
            try:
                roc_auc = roc_auc_score(all_targets, all_probabilities[:, 1])
                logging.info(f'Epoch {epoch+1}/{num_epochs} Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%, AUC-ROC: {roc_auc:.4f}')
            except Exception as e:
                logging.info(f'Epoch {epoch+1}/{num_epochs} Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')
        else:
            logging.info(f'Epoch {epoch+1}/{num_epochs} Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')
        
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            best_model_state = model.state_dict().copy()
            logging.info(f"New best model at epoch {epoch+1} with Val Accuracy: {best_val_acc:.2f}%")
    
    total_time = time.time() - start_time
    logging.info(f"Total training time: {total_time:.2f} seconds")
    
    # Load best model for testing
    if best_model_state:
        model.load_state_dict(best_model_state)
        logging.info(f"Loaded best model with Val Accuracy: {best_val_acc:.2f}%")
    
    model.eval()
    epoch_test_loss = 0.0
    correct_test = 0
    total_test_samples = 0
    all_targets = []
    all_predictions = []
    all_probabilities = []
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            epoch_test_loss += loss.item() * data.size(0)
            total_test_samples += data.size(0)
            _, predicted = torch.max(output, 1)
            correct_test += (predicted == target).sum().item()
            all_targets.extend(target.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            all_probabilities.extend(F.softmax(output, dim=1).cpu().numpy())
    
    test_loss = epoch_test_loss / total_test_samples
    test_accuracy = 100.0 * correct_test / total_test_samples
    
    if num_classes == 2:
        try:
            test_auc = roc_auc_score(np.array(all_targets), np.array(all_probabilities)[:, 1])
        except Exception as e:
            logging.error(f"Error calculating Test AUC-ROC: {e}")
            test_auc = None
    else:
        test_auc = None
    
    logging.info("\n===== Final Test Results =====")
    logging.info(f"Test Loss: {test_loss:.4f}")
    logging.info(f"Test Accuracy: {test_accuracy:.2f}%")
    if test_auc is not None:
        logging.info(f"Test AUC-ROC: {test_auc:.4f}")
    
    # Save the trained model
    os.makedirs('models', exist_ok=True)
    model_filename = "models/training_using_gpus_1_model.pth"
    torch.save(model.state_dict(), model_filename)
    logging.info(f"Model saved to {model_filename}")
    
    # Plot training history
    epochs_range = range(1, num_epochs+1)
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, train_losses, label='Train Loss')
    plt.plot(epochs_range, val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss Curves')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, train_accuracies, label='Train Accuracy')
    plt.plot(epochs_range, val_accuracies, label='Val Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Accuracy Curves')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    os.makedirs('plots', exist_ok=True)
    plot_filename = "plots/training_using_gpus_1_results.png"
    plt.savefig(plot_filename)
    plt.close()
    logging.info(f"Training plots saved as {plot_filename}")
    
    # Save runtime parameters and metrics as JSON
    result = {
        "test_accuracy": test_accuracy,
        "test_auc_roc": test_auc,
        "computing_time": total_time,
        "train_losses": train_losses,
        "train_accuracies": train_accuracies,
        "val_losses": val_losses,
        "val_accuracies": val_accuracies,
        "hyperparameters": {
            "batch_size": batch_size,
            "num_epochs": num_epochs,
            "initial_lr": initial_lr,
            "weight_decay": weight_decay,
            "num_classes": num_classes
        },
        "total_parameters": total_params,
        "total_layers": total_layers
    }
    os.makedirs('metrics', exist_ok=True)
    params_filename = "metrics/training_using_gpus_1_params.json"
    with open(params_filename, 'w') as f:
        json.dump(result, f, indent=4)
    logging.info(f"Runtime parameters saved as {params_filename}")
    
    return result

#### Main function to handle and call all the functional flow

In [4]:
def main():
    hostname = socket.gethostname()
    logging.info(f"Running on node: {hostname}")
    data_dir = "../../preprocessed_glaucoma_data" 
    logging.info("Training Streamlined Medical CNN model for Glaucoma for 10 epochs on a single GPU...")
    results = train_and_evaluate(data_dir)
    logging.info("Training completed.")
    logging.info(f"Test Accuracy: {results['test_accuracy']:.2f}%")
    logging.info(f"Total Layers: {results['total_layers']}")
    logging.info(f"Total Parameters: {results['total_parameters']:,}")

if __name__ == "__main__":
    main()

2025-04-04 17:26:16,180 - INFO - Running on node: d1004
2025-04-04 17:26:16,180 - INFO - Training Streamlined Medical CNN model for Glaucoma for 10 epochs on a single GPU...
2025-04-04 17:26:16,186 - INFO - Loading data...
2025-04-04 17:26:43,057 - INFO - Data shapes: Train (23898, 224, 224, 3), Val (5747, 224, 224, 3), Test (2874, 224, 224, 3)
2025-04-04 17:26:43,114 - INFO - Using device: cuda
2025-04-04 17:26:43,720 - INFO - Model architecture: MedicalCNN
2025-04-04 17:26:43,721 - INFO - Total layers: 129
2025-04-04 17:26:43,721 - INFO - Total parameters: 22,494,274
2025-04-04 17:26:44,930 - INFO - Epoch [1/10], Batch [0/747], Loss: 0.7060
2025-04-04 17:26:45,573 - INFO - Epoch [1/10], Batch [10/747], Loss: 0.6762
2025-04-04 17:26:46,203 - INFO - Epoch [1/10], Batch [20/747], Loss: 0.7399
2025-04-04 17:26:46,833 - INFO - Epoch [1/10], Batch [30/747], Loss: 0.7481
2025-04-04 17:26:47,460 - INFO - Epoch [1/10], Batch [40/747], Loss: 0.6709
2025-04-04 17:26:48,090 - INFO - Epoch [1/10]

When training on the V100-SXM2 GPU, the process was much faster compared to a CPU. The GPU’s ability to handle parallel processing really made a difference here, unlike CPUs that perform tasks sequentially, the GPU was able to run many operations at the same time (like matrix multiplications and convolutions), which significantly sped up training.

On a CPU, the model has taken about 12.6 hours to train. However, with the V100-SXM2 GPU, I saw a notable reduction in training time. This showcases how much faster deep learning tasks can be with GPUs, especially for models with a large number of layers like mine.